In [6]:
import os
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import time
import random

# Defining the State columns
columns = ['ALT(m)', 'Phi(deg)', 'Theta(deg)', 'Psi(deg)', 'Vx(m/s)', 'Vy(m/s)', 'Vz(m/s)', 'P(deg/s)', 'Q(deg/s)', 'R(deg/s)', 'Nx(m/s2)', 'Ny(m/s2)', 'Nz(m/s2)', 'Radial(deg)', 'Distance(m)', 'DeltaAlt:Anv-Tgt(m)']

# Function to set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

num_synthetic_flights_list = [7, 15, 30, 45, 60, 150]

for num_synthetic_flights in num_synthetic_flights_list:
    # Function to load and process flight data
    def load_and_process_flight_data(file_paths):
        flights = []
        for file_path in file_paths:
            df = pd.read_csv(file_path, delimiter='\t')
            if 'Time(milli)' in df.columns:
                df = df.drop(columns=['Time(milli)'])
            flights.append(df)
        return flights

    # Function to create subsequences from flight data for training
    def create_subsequences(flight_data, seq_length):
        state_subsequences = []
        action_subsequences = []
        flight_ids = []
        for flight_id, df in enumerate(flight_data):
            states =  df[columns].values
            actions = df[['JX', 'JY', 'Throttle']].values
            num_subsequences = len(states) - seq_length + 1
            for i in range(num_subsequences):
                state_subsequence = states[i:i+seq_length]
                action_subsequence = actions[i:i+seq_length]  # Now a sequence
                state_subsequences.append(state_subsequence)
                action_subsequences.append(action_subsequence)
                flight_ids.append(flight_id)
        return np.array(state_subsequences), np.array(action_subsequences), np.array(flight_ids)

    # Function to plot actual vs predicted actions with mean and standard deviation
    def plot_trajectory_with_mean_std(actual_actions_list, predicted_actions_list, plots_dir):
        actual_actions = np.array(actual_actions_list)
        predicted_actions = np.array(predicted_actions_list)

        # Adjust time steps to handle different lengths
        max_time_steps = max([actions.shape[0] for actions in actual_actions_list])
        time_steps = np.arange(max_time_steps)

        # Pad sequences to the same length
        actual_padded = np.array([np.pad(actions, ((0, max_time_steps - actions.shape[0]), (0, 0)), 'edge') for actions in actual_actions_list])
        predicted_padded = np.array([np.pad(actions, ((0, max_time_steps - actions.shape[0]), (0, 0)), 'edge') for actions in predicted_actions_list])

        # Calculate mean and std for actual and predicted actions
        actual_mean = np.mean(actual_padded, axis=0)
        actual_std = np.std(actual_padded, axis=0)
        predicted_mean = np.mean(predicted_padded, axis=0)
        predicted_std = np.std(predicted_padded, axis=0)

        plt.figure(figsize=(15, 5))

        # Plot JX
        plt.subplot(1, 3, 1)
        plt.plot(time_steps, actual_mean[:, 0], label='Actual JX (Mean)', color='blue')
        plt.fill_between(time_steps, actual_mean[:, 0] - actual_std[:, 0], actual_mean[:, 0] + actual_std[:, 0], color='blue', alpha=0.2, label='Actual JX (± Std)')
        plt.plot(time_steps, predicted_mean[:, 0], label='Predicted JX (Mean)', linestyle='--', color='red')
        plt.fill_between(time_steps, predicted_mean[:, 0] - predicted_std[:, 0], predicted_mean[:, 0] + predicted_std[:, 0], color='red', alpha=0.2, label='Predicted JX (± Std)')
        plt.xlabel('Time Step')
        plt.ylabel('JX')
        plt.legend(loc='lower center', fontsize='x-small', ncol=2, handlelength=2.5, handletextpad=1.5)

        # Plot JY
        plt.subplot(1, 3, 2)
        plt.plot(time_steps, actual_mean[:, 1], label='Actual JY (Mean)', color='blue')
        plt.fill_between(time_steps, actual_mean[:, 1] - actual_std[:, 1], actual_mean[:, 1] + actual_std[:, 1], color='blue', alpha=0.2, label='Actual JY (± Std)')
        plt.plot(time_steps, predicted_mean[:, 1], label='Predicted JY (Mean)', linestyle='--', color='red')
        plt.fill_between(time_steps, predicted_mean[:, 1] - predicted_std[:, 1], predicted_mean[:, 1] + predicted_std[:, 1], color='red', alpha=0.2, label='Predicted JY (± Std)')
        plt.xlabel('Time Step')
        plt.ylabel('JY')
        plt.legend(loc='lower center', fontsize='x-small', ncol=2, handlelength=2.5, handletextpad=1.5)

        # Plot Throttle
        plt.subplot(1, 3, 3)
        plt.plot(time_steps, actual_mean[:, 2], label='Actual Throttle (Mean)', color='blue')
        plt.fill_between(time_steps, actual_mean[:, 2] - actual_std[:, 2], actual_mean[:, 2] + actual_std[:, 2], color='blue', alpha=0.2, label='Actual Throttle (± Std)')
        plt.plot(time_steps, predicted_mean[:, 2], label='Predicted Throttle (Mean)', linestyle='--', color='red')
        plt.fill_between(time_steps, predicted_mean[:, 2] - predicted_std[:, 2], predicted_mean[:, 2] + predicted_std[:, 2], color='red', alpha=0.2, label='Predicted Throttle (± Std)')
        plt.xlabel('Time Step')
        plt.ylabel('Throttle')
        plt.legend(loc='lower center', fontsize='x-small', ncol=2, handlelength=2.5, handletextpad=1.5)

        plt.suptitle('Trajectory Comparison: Actual vs Predicted with Mean and Standard Deviation')
        plt.tight_layout()
        plt.savefig(os.path.join(plots_dir, 'trajectory_comparison_mean_std.png'), format='png', dpi=500)
        plt.close()

    # Directory containing the adjusted flight data files
    adjusted_data_directory = './data/adjusted_flights/'
    adjusted_file_pattern = os.path.join(adjusted_data_directory, 'SimuladorDeVoo_*.txt')
    adjusted_files = glob.glob(adjusted_file_pattern)

    # Load and process the adjusted flight data
    flight_data_adjusted = load_and_process_flight_data(adjusted_files)

    # Split adjusted flights into 70% training/validation and 30% test
    flight_data_train_val_adjusted, flight_data_test = train_test_split(flight_data_adjusted, test_size=0.30, random_state=42)

    # Directory containing the generated flight data files
    generated_data_directory = f'./data/smoothed_flights_{num_synthetic_flights}/'
    generated_file_pattern = os.path.join(generated_data_directory, 'GeneratedFlight_*.txt')
    generated_files = glob.glob(generated_file_pattern)

    # Load and process the generated flight data
    flight_data_generated = load_and_process_flight_data(generated_files)

    # Combine the 80% adjusted training/validation data with all generated data
    flight_data_train_val = flight_data_train_val_adjusted + flight_data_generated

    print(f'Training/validation set has {len(flight_data_train_val)} samples.')
    print(f'Test set has {len(flight_data_test)} samples from adjusted flights only.')

    # Define sequence length for training
    sequence_length = 5

    # Create subsequences from flight data for training
    state_subsequences_train_val, action_subsequences_train_val, flight_ids_train_val = create_subsequences(flight_data_train_val, sequence_length)

    # Create subsequences from flight data for test
    state_subsequences_test, action_subsequences_test, flight_ids_test = create_subsequences(flight_data_test, sequence_length)

    # Set random seed for cross-validation
    cv_seed = 42
    set_seed(cv_seed)

    # K-Fold Cross Validation on training/validation data
    # Get unique flight IDs
    unique_flight_ids = np.unique(flight_ids_train_val)

    kf = KFold(n_splits=5, shuffle=True, random_state=cv_seed)

    fold_no = 1
    metrics_cv_list = []
    epochs_per_fold = []

    # Directories to save metrics, models, and plots
    metrics_dir = f'./metrics/lstm-full-augmented_{num_synthetic_flights}/'
    plots_dir = f'./plots/lstm-full-augmented_{num_synthetic_flights}/'
    models_dir = f'./models/lstm-full-augmented_{num_synthetic_flights}/'
    os.makedirs(metrics_dir, exist_ok=True)
    os.makedirs(plots_dir, exist_ok=True)
    os.makedirs(models_dir, exist_ok=True)

    for train_flight_ids_idx, val_flight_ids_idx in kf.split(unique_flight_ids):
        # Get flight IDs for training and validation
        train_flight_ids = unique_flight_ids[train_flight_ids_idx]
        val_flight_ids = unique_flight_ids[val_flight_ids_idx]

        # Map flight IDs to indices in subsequences
        train_indices = np.isin(flight_ids_train_val, train_flight_ids)
        val_indices = np.isin(flight_ids_train_val, val_flight_ids)

        # Select subsequences corresponding to training and validation flights
        states_train, actions_train = state_subsequences_train_val[train_indices], action_subsequences_train_val[train_indices]
        states_val, actions_val = state_subsequences_train_val[val_indices], action_subsequences_train_val[val_indices]

        # Normalize the data using training data mean and std
        state_mean = np.mean(states_train, axis=(0,1))
        state_std = np.std(states_train, axis=(0,1))
        states_train = (states_train - state_mean) / state_std
        states_val = (states_val - state_mean) / state_std  # Use training mean and std

        # Normalize the actions using training data mean and std
        action_mean = np.mean(actions_train, axis=(0,1))
        action_std = np.std(actions_train, axis=(0,1))
        actions_train = (actions_train - action_mean) / action_std
        actions_val = (actions_val - action_mean) / action_std  # Use training action mean and std

        # Define the model
        model = Sequential()
        model.add(LSTM(128, input_shape=(sequence_length, states_train.shape[2]), return_sequences=True))
        model.add(TimeDistributed(Dense(64, activation='relu')))
        model.add(TimeDistributed(Dense(32, activation='relu')))
        model.add(TimeDistributed(Dense(actions_train.shape[2])))

        # Compile the model
        optimizer = Adam(learning_rate=0.0001)
        model.compile(optimizer=optimizer, loss='mse')

        # Define early stopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

        # Measure training time
        train_start_time = time.time()

        # Train the model with validation and early stopping
        history = model.fit(states_train, actions_train, epochs=int(1e6), batch_size=64, 
                            validation_data=(states_val, actions_val), callbacks=[early_stopping], verbose=0)

        train_end_time = time.time()
        training_time = train_end_time - train_start_time

        # Record the number of epochs used
        epochs_per_fold.append(len(history.history['loss']))

        # Measure inference time
        inference_start_time = time.time()

        # Evaluate the model on the validation set
        val_loss = model.evaluate(states_val, actions_val, verbose=0)

        # Predict actions on the validation set
        predicted_actions = model.predict(states_val, verbose=0)

        inference_end_time = time.time()
        inference_time = inference_end_time - inference_start_time

        print(f'Fold {fold_no} - Training Time: {training_time:.4f} seconds, Inference Time: {inference_time:.4f} seconds')

        # Reshape for metrics calculations
        actions_val_reshaped = actions_val.reshape(-1, actions_val.shape[2])
        predicted_actions_reshaped = predicted_actions.reshape(-1, predicted_actions.shape[2])

        # Denormalize the actions for evaluation
        actions_val_denorm = actions_val_reshaped * action_std + action_mean
        predicted_actions_denorm = predicted_actions_reshaped * action_std + action_mean

        # Calculate MSE, RMSE, MAE, R²
        mse = mean_squared_error(actions_val_denorm, predicted_actions_denorm)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(actions_val_denorm, predicted_actions_denorm)
        r2 = r2_score(actions_val_denorm, predicted_actions_denorm)

        # Append metrics and timing to list
        metrics_cv_list.append({
            'fold': fold_no,
            'loss': val_loss,
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'r2': r2,
            'training_time': training_time,
            'inference_time': inference_time
        })

        print(f'Fold {fold_no} - Validation MSE: {mse}')
        print(f'Fold {fold_no} - Validation RMSE: {rmse}')
        print(f'Fold {fold_no} - Validation MAE: {mae}')
        print(f'Fold {fold_no} - Validation R²: {r2}')

        # Plot training & validation loss values
        plt.figure(figsize=(10, 5))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(f'Model Loss - Fold {fold_no}')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(loc='upper right')
        plt.savefig(os.path.join(plots_dir, f'loss_fold_{fold_no}.png'))
        plt.close()

        # Save the model
        model.save(os.path.join(models_dir, f'model_fold_{fold_no}.keras'))

        fold_no += 1

    # Calculate the average number of epochs from cross-validation
    avg_epochs = int(np.mean(epochs_per_fold))
    print(f'Average number of epochs from cross-validation: {avg_epochs}')

    # Convert metrics_cv_list to DataFrame
    metrics_cv_df = pd.DataFrame(metrics_cv_list)

    # Calculate mean and std for each metric
    mean_metrics_cv = metrics_cv_df.mean(numeric_only=True)
    std_metrics_cv = metrics_cv_df.std(numeric_only=True)

    # Add the fold column for mean and std
    mean_metrics_cv['fold'] = 'mean'
    std_metrics_cv['fold'] = 'std'

    # Convert mean and std to DataFrame
    mean_metrics_cv_df = pd.DataFrame([mean_metrics_cv])
    std_metrics_cv_df = pd.DataFrame([std_metrics_cv])

    # Append mean and std rows to the metrics_cv_df
    metrics_cv_df = pd.concat([metrics_cv_df, mean_metrics_cv_df, std_metrics_cv_df], ignore_index=True)

    # Save cross-validation metrics DataFrame to CSV
    metrics_cv_df.to_csv(os.path.join(metrics_dir, 'cross_validation_metrics.csv'), index=False)

    # Now, train the final model using different seeds
    final_model_metrics_list = []

    seeds = [43, 44, 45, 46, 47]

    for seed in seeds:
        set_seed(seed)

        # Normalize the full training data using its own mean and std
        state_mean_full = np.mean(state_subsequences_train_val, axis=(0,1))
        state_std_full = np.std(state_subsequences_train_val, axis=(0,1))
        state_subsequences_train_val_normalized = (state_subsequences_train_val - state_mean_full) / state_std_full

        action_mean_full = np.mean(action_subsequences_train_val, axis=(0,1))
        action_std_full = np.std(action_subsequences_train_val, axis=(0,1))
        action_subsequences_train_val_normalized = (action_subsequences_train_val - action_mean_full) / action_std_full

        # Train the final model using the average number of epochs on the full training/validation data
        final_model = Sequential()
        final_model.add(LSTM(128, input_shape=(sequence_length, state_subsequences_train_val_normalized.shape[2]), return_sequences=True))
        final_model.add(TimeDistributed(Dense(64, activation='relu')))
        final_model.add(TimeDistributed(Dense(32, activation='relu')))
        final_model.add(TimeDistributed(Dense(action_subsequences_train_val_normalized.shape[2])))

        # Compile the final model
        optimizer = Adam(learning_rate=0.0001)
        final_model.compile(optimizer=optimizer, loss='mse')

        # Measure training time
        train_start_time = time.time()

        # Train the final model for avg_epochs without early stopping
        final_model.fit(state_subsequences_train_val_normalized, action_subsequences_train_val_normalized, epochs=avg_epochs, batch_size=64, verbose=0)

        train_end_time = time.time()
        final_training_time = train_end_time - train_start_time

        print(f'Final Model (Seed {seed}) - Training Time: {final_training_time:.4f} seconds')

        # Normalize the test data using the training data mean and std
        state_subsequences_test_normalized = (state_subsequences_test - state_mean_full) / state_std_full
        action_subsequences_test_normalized = (action_subsequences_test - action_mean_full) / action_std_full

        # Measure inference time
        inference_start_time = time.time()

        # Evaluate the final model on the test set
        final_loss = final_model.evaluate(state_subsequences_test_normalized, action_subsequences_test_normalized, verbose=0)

        # Predict actions on the test set using the final model
        final_predicted_actions = final_model.predict(state_subsequences_test_normalized, verbose=0)

        inference_end_time = time.time()
        final_inference_time = inference_end_time - inference_start_time

        print(f'Final Model (Seed {seed}) - Inference Time: {final_inference_time:.4f} seconds')

        # Reshape for metrics calculations
        actions_test_reshaped = action_subsequences_test_normalized.reshape(-1, action_subsequences_test_normalized.shape[2])
        final_predicted_actions_reshaped = final_predicted_actions.reshape(-1, final_predicted_actions.shape[2])

        # Denormalize actions for evaluation
        actions_test_denorm = actions_test_reshaped * action_std_full + action_mean_full
        final_predicted_actions_denorm = final_predicted_actions_reshaped * action_std_full + action_mean_full

        # Calculate metrics for the final model on test data
        final_mse = mean_squared_error(actions_test_denorm, final_predicted_actions_denorm)
        final_rmse = np.sqrt(final_mse)
        final_mae = mean_absolute_error(actions_test_denorm, final_predicted_actions_denorm)
        final_r2 = r2_score(actions_test_denorm, final_predicted_actions_denorm)

        # Append final model metrics and timing to list
        final_model_metrics_list.append({
            'seed': seed,
            'loss': final_loss,
            'mse': final_mse,
            'rmse': final_rmse,
            'mae': final_mae,
            'r2': final_r2,
            'training_time': final_training_time,
            'inference_time': final_inference_time
        })

        print(f'Final Model (Seed {seed}) Metrics:')
        print(f'Test MSE: {final_mse}')
        print(f'Test RMSE: {final_rmse}')
        print(f'Test MAE: {final_mae}')
        print(f'Test R²: {final_r2}')

        # Save the final model
        final_model.save(os.path.join(models_dir, f'final_model_seed_{seed}.keras'))

    # After the loop over seeds, process the final model metrics
    final_model_metrics_df = pd.DataFrame(final_model_metrics_list)

    # Calculate mean and std for each metric
    mean_final_model_metrics = final_model_metrics_df.mean(numeric_only=True)
    std_final_model_metrics = final_model_metrics_df.std(numeric_only=True)

    # Add the seed column for mean and std
    mean_final_model_metrics['seed'] = 'mean'
    std_final_model_metrics['seed'] = 'std'

    # Convert mean and std to DataFrame
    mean_final_model_metrics_df = pd.DataFrame([mean_final_model_metrics])
    std_final_model_metrics_df = pd.DataFrame([std_final_model_metrics])

    # Append mean and std rows to the final_model_metrics_df
    final_model_metrics_df = pd.concat([final_model_metrics_df, mean_final_model_metrics_df, std_final_model_metrics_df], ignore_index=True)

    # Save final model metrics DataFrame to CSV
    final_model_metrics_df.to_csv(os.path.join(metrics_dir, 'final_model_metrics.csv'), index=False)

    # Optionally, plot the trajectory comparison using one of the final models (e.g., the last one)
    print("Predicting the test dataset using the final model...")

    # Prepare test sequences for full trajectory prediction
    state_sequences_test = []
    action_sequences_test = []

    for df in flight_data_test:
        states =  df[columns].values
        actions = df[['JX', 'JY', 'Throttle']].values
        state_sequences_test.append(states)
        action_sequences_test.append(actions)

    # Use the last trained final model for prediction
    final_model = final_model  # The last model trained in the loop

    # Predict full flight trajectories using the final model
    actual_actions_list = []
    predicted_actions_list = []

    for i, state_sequence in enumerate(state_sequences_test):
        predicted_actions_full = []

        num_steps = len(state_sequence)
        for j in range(sequence_length - 1, num_steps):
            window = state_sequence[j - sequence_length + 1:j + 1]
            # Normalize window
            window_normalized = (window - state_mean_full) / state_std_full
            window_normalized = window_normalized.reshape(1, sequence_length, state_sequence.shape[1])
            predicted_action = final_model.predict(window_normalized, verbose=0)
            # Denormalize the predicted action
            predicted_action_denorm = predicted_action[0, -1] * action_std_full + action_mean_full
            predicted_actions_full.append(predicted_action_denorm)

        # Pad the beginning to match the original sequence length
        for _ in range(sequence_length - 1):
            predicted_actions_full.insert(0, predicted_actions_full[0])

        predicted_actions_full = np.array(predicted_actions_full)
        actual_actions_full = action_sequences_test[i][:len(predicted_actions_full)]  # Align lengths

        actual_actions_list.append(actual_actions_full)
        predicted_actions_list.append(predicted_actions_full)
        print(f'Full flight trajectory {i+1} predicted and collected.')

    # Plot the mean and standard deviation for the collected trajectories
    plot_trajectory_with_mean_std(actual_actions_list, predicted_actions_list, plots_dir)
    print('Trajectory comparison plot with mean and standard deviation saved.')

Training/validation set has 28 samples.
Test set has 9 samples from adjusted flights only.
Fold 1 - Training Time: 108.4406 seconds, Inference Time: 38.2363 seconds
Fold 1 - Validation MSE: 5.162710661485419
Fold 1 - Validation RMSE: 2.272159911072594
Fold 1 - Validation MAE: 0.92762533741233
Fold 1 - Validation R²: 0.949468491967195
Fold 2 - Training Time: 60.5526 seconds, Inference Time: 0.7836 seconds
Fold 2 - Validation MSE: 5.545190481081183
Fold 2 - Validation RMSE: 2.354822813096812
Fold 2 - Validation MAE: 1.00664561223553
Fold 2 - Validation R²: 0.949086292699301
Fold 3 - Training Time: 100.2915 seconds, Inference Time: 0.7834 seconds
Fold 3 - Validation MSE: 5.614115571962902
Fold 3 - Validation RMSE: 2.369412495105675
Fold 3 - Validation MAE: 0.9857292570883002
Fold 3 - Validation R²: 0.9466801047267546
Fold 4 - Training Time: 65.4652 seconds, Inference Time: 20.4895 seconds
Fold 4 - Validation MSE: 6.1080192647908715
Fold 4 - Validation RMSE: 2.4714407265380394
Fold 4 - Val